# 1. CONCEITOS FUNDAMENTAIS

Em programação paralela, o objetivo é dividir o trabalho de modo a que várias unidades de processamento (processos ou cores) executem partes do problema em simultâneo.

Essa divisão pode ocorrer de duas formas principais:

**Distribuição espacial (ou decomposição de dados)**

Cada processo trata uma parte diferente dos dados, executando o mesmo algoritmo sobre subconjuntos distintos.

> Paradigma dominante em HPC (High Performance Computing).

> Permite escalar problemas muito grandes (imagens, matrizes, simulações).

> Base das operações Scatter, Gather, Allreduce e Gatherv do MPI.

Exemplo:

Uma matriz 1000×1000 pode ser dividida em blocos de 250×250.

Cada processo calcula o seu bloco e, no fim, os resultados são reunidos.

**Distribuição temporal (ou paralelismo de tarefas)**

Divide-se o trabalho em fases sucessivas de uma mesma tarefa.

Cada processo executa uma fase diferente, e os resultados fluem em cadeia — conceito conhecido como pipeline.

> Usado em streaming, processamento de áudio/vídeo em tempo real ou simulações com passos de tempo.

> O trabalho de cada processo depende do anterior (comunicação ponto-a-ponto).

Exemplo:

 - Processo 0 lê dados de um sensor.

 - Processo 1 processa esses dados.

 - Processo 2 grava resultados no disco.
 
Enquanto o processo 1 está a processar, o 0 já pode ler o próximo bloco.

As comunicações não bloqueantes (Isend, Irecv) permitem fluxo contínuo (overlap entre leitura, cálculo e escrita).Um problema pode ser dividido segundo duas dimensões principais: espacial (dados) e temporal (tarefas).

# 2. MODELOS DE PARALELISMO

| Modelo | Tipo | Características | Exemplos |
|:--------|:------|:-----------------|:----------|
|Data Parallelism |Espacial |Mesma operação em subconjuntos de dados |processamento de imagem, áudio, simulação de partículas|
|Task Parallelism|Temporal|Tarefas diferentes sobre o mesmo ou diferentes dados|pipeline, workflow científico|
|Pipeline Parallelism|Temporal (sequencial em etapas)|Saída de uma tarefa é entrada da seguinte|processamento de frames, deep learning inference|
|Hybrid|Combina ambos|Cada nó executa várias tarefas sobre uma subparte dos dados|simulações complexas, IA distribuída|

# 3. DECOMPOSIÇÃO DE PROBLEMAS

Um problema pode ser dividido segundo duas dimensões principais: espacial (dados) e temporal (tarefas).

**a) Decomposição Espacial (de dados)**

> **Divisão geométrica:** domínio físico ou matemático (grelha, matriz, volume 3D).

Exemplo: numa simulação 2D, cada processo calcula uma submatriz.
   
> **Divisão lógica:** conjuntos independentes (ficheiros, blocos de áudio, registos).

Exemplo: cada processo processa um ficheiro .wav diferente.
   
Em HPC, este tipo de decomposição é o mais comum — cada core recebe um subconjunto de dados, processa localmente e comunica com os outros via MPI.

Funções relevantes do MPI:

 - Scatter / Scatterv → distribuição dos dados;

 - Gather / Gatherv → recolha dos resultados;

 - Allreduce → combinação de valores (somatório, média, etc.).
   
**b) Decomposição Temporal (de tarefas)**

Divide o trabalho em etapas sequenciais, cada uma realizada por um processo distinto.
É uma forma de task parallelism.

 - Cada processo executa uma fase específica;

 - As fases comunicam entre si em série, normalmente via Send / Recv ou versões assíncronas Isend / Irecv;

 - Permite *overlap* temporal — enquanto um processo processa, outro já prepara a próxima entrada.

Usos típicos:

 - Processamento em streaming (áudio, vídeo, sensores);

 - Simulações dependentes do tempo;

 - Pipelines de inferência em modelos de IA.
    
*MPI aplica-se aqui: Isend, Irecv (não bloqueantes) permitem fluxo contínuo.*


# 4. RESUMO

| Tipo de decomposição   | Comunicação entre processos             | Objetivo                              |
| ---------------------- | --------------------------------------- | ------------------------------------- |
| **Espacial (dados)**   | Distribuição e recolha (Scatter/Gather) | Processar subconjuntos em paralelo    |
| **Temporal (tarefas)** | Fluxo em série (Send/Recv)              | Sobrepor fases diferentes da execução |

# 5. COMUNICAÇÃO E MODELOS DE PARALELISMO

Os dois grandes modelos de paralelismo — espacial (de dados) e temporal (de tarefas) — concretizam-se, em MPI, através de diferentes tipos de comunicação entre processos.
A forma como os processos trocam informação define a eficiência e a escalabilidade da aplicação.

### 5.1 Comunicações ponto-a-ponto e coletivas

**Comunicações ponto-a-ponto (point-to-point)**

> São usadas quando os processos trocam dados diretamente entre si, de forma explícita e controlada.
Estão associadas ao paralelismo temporal e a pipelines de tarefas, onde a sequência de operações importa.
Exemplo típico: Send, Recv, Isend, Irecv.

Aplicações:

 - Fluxos encadeados (pipelines);

 - Processamento em tempo real (áudio, sensores, vídeo);

 - Simulações dependentes do tempo, com dependências entre passos.
 
**Comunicações coletivas (collective)**

> Envolvem todo o grupo de processos (COMM_WORLD ou sub-comunicadores) e distribuem ou reúnem dados de forma global.
São típicas do paralelismo espacial, onde os dados estão divididos e cada processo contribui com uma parte.
Exemplo típico: Scatter, Gather, Allreduce, Bcast.

Aplicações:

 - Processamento de grandes matrizes ou vetores;

 - Simulações físicas distribuídas em grelhas;

 - Redução de resultados (somas, médias, máximos).
 
**Em resumo:**

 - Espacial → comunicação coletiva (dados partilhados globalmente).

 - Temporal → comunicação ponto-a-ponto (dados encadeados entre tarefas).
 
**Nota sobre deadlock**

> Um deadlock (ou impasse) ocorre quando dois ou mais processos ficam bloqueados à espera uns dos outros, impedindo o programa de prosseguir.
Em MPI isto acontece, por exemplo, se dois processos chamarem Send em simultâneo sem que nenhum tenha executado o Recv correspondente — ambos aguardam indefinidamente.
>
> Para evitar este bloqueio:
>
> - alterna a ordem das chamadas (Send num lado, Recv no outro);
>
> - ou utiliza Sendrecv, que envia e recebe numa única operação;
>
> - ou recorre a operações não bloqueantes (Isend, Irecv) seguidas de Wait.

Estas estratégias garantem que a comunicação flui sem esperas circulares.


 
### 5.2 Comunicações bloqueantes e não bloqueantes

A eficiência do paralelismo depende de como os processos esperam (ou não) pela comunicação:

 - Bloqueantes (Send, Recv, Bcast, Gather, etc.). O processo espera que a transferência termine antes de continuar. São mais simples e seguras, adequadas a modelos espaciais com sincronização forte — por exemplo, quando todos terminam uma etapa antes da próxima.

 - Não bloqueantes (Isend, Irecv, Iallreduce, etc.). A comunicação é iniciada, mas o processo pode continuar a computação em paralelo. Usadas em modelos temporais (pipelines ou fluxos contínuos), onde é desejável que leitura, cálculo e escrita ocorram em simultâneo. Permitem overlap entre comunicação e cálculo.
 
### 5.3 Overlapping (computação + comunicação)

O overlapping surge naturalmente no paralelismo temporal, quando cada processo realiza simultaneamente partes diferentes da execução.

 - Exemplo: enquanto um processo está a enviar dados do passo t, outro já processa o passo t+1.

 - Em MPI, obtém-se *overlap* com operações não bloqueantes (Isend, Irecv) e sincronizações seletivas (Wait, Test).

 - Nos modelos espaciais, o *overlap* é menos frequente, mas pode ser explorado para trocar fronteiras de domínios enquanto se calcula o interior (método usado em simulações numéricas).

O objetivo é reduzir o tempo ocioso dos processos, evitando esperas desnecessárias entre fases de comunicação e cálculo.

### 5.4 Comunicação com arrays vs objetos Python

A forma como os dados são representados influencia o desempenho:

 - Modelos espaciais lidam com grandes volumes de dados numéricos (matrizes, vetores). Nesses casos, deve-se usar arrays NumPy ou outros objetos que exponham buffers de memória contíguos, para que o MPI transfira os dados diretamente, sem pickle.
 
>Exemplos: comm.Scatter(array, ...), comm.Gather(array, ...).

 - Modelos temporais frequentemente lidam com objetos heterogéneos ou mensagens pequenas (strings, estados, pacotes). Aí é mais prático usar comm.send e comm.recv, que funcionam com qualquer objeto Python (serializado internamente).

>Exemplo: comm.send(msg, dest=1) / comm.recv(source=0).

Regra prática:

 - Paralelismo espacial → arrays NumPy (eficiência máxima).

 - Paralelismo temporal → objetos Python (flexibilidade).
 
### 5.5 Síntese das relações entre conceitos e tipos de paralelismo

A escolha entre paralelismo espacial e temporal define o tipo de comunicação usada, a forma de sincronização e até o formato dos dados trocados.
Abaixo apresentam-se os conceitos e mecanismos típicos de cada modelo.

### Paralelismo Espacial (Decomposição de Dados)

| Conceito                    | Tipo de Comunicação | Natureza       | Objetivo                                                       | Funções MPI Típicas   | Estrutura de Dados |
| --------------------------- | ------------------- | -------------- | -------------------------------------------------------------- | --------------------- | ------------------ |
| **Distribuição de dados**   | **Coletiva**        | **Bloqueante** | Dividir dados globais entre processos                          | `Scatter`, `Scatterv` | `numpy.ndarray`    |
| **Recolha de resultados**   | **Coletiva**        | **Bloqueante** | Reunir resultados parciais                                     | `Gather`, `Gatherv`   | `numpy.ndarray`    |
| **Redução global**          | **Coletiva**        | **Bloqueante** | Combinar valores (soma, média, etc.)                           | `Reduce`, `Allreduce` | `numpy.ndarray`    |
| **Sincronização de etapas** | **Coletiva**        | **Bloqueante** | Garantir que todos os processos terminam antes da próxima fase | `Barrier`             | —                  |

Características principais:

 - Todos os processos executam o mesmo algoritmo sobre subconjuntos distintos de dados.

 - Requer sincronização global em cada iteração.

 - Comunicações são bloqueantes por natureza.

 - O desempenho depende da distribuição equilibrada dos dados.
 
### Paralelismo Temporal (Decomposição de Tarefas)

| Conceito                       | Tipo de Comunicação | Natureza                          | Objetivo                                   | Funções MPI Típicas          | Estrutura de Dados                       |
| ------------------------------ | ------------------- | --------------------------------- | ------------------------------------------ | ---------------------------- | ---------------------------------------- |
| **Encadeamento de tarefas**    | **Ponto-a-ponto**   | **Não bloqueante**                | Passar resultados entre etapas do pipeline | `Isend`, `Irecv`             | Objetos Python (mensagens pequenas)      |
| **Fluxo contínuo (streaming)** | **Ponto-a-ponto**   | **Não bloqueante**                | Sobrepor leitura, cálculo e escrita        | `Isend`, `Irecv`, `Wait`     | Buffers ou blocos de dados               |
| **Comunicação sequencial**     | **Ponto-a-ponto**   | **Bloqueante** (em casos simples) | Transmissão direta sem sobreposição        | `Send`, `Recv`               | Objetos simples (texto, estado, comando) |
| **Coordenação entre fases**    | **Ponto-a-ponto**   | **Assíncrona**                    | Sincronizar o pipeline sem paragens totais | `Test`, `Waitany`, `Barrier` | —                                        |

Características principais:

 - Cada processo executa uma fase distinta do processamento.

 - As fases estão encadeadas — a saída de uma é a entrada da seguinte.

 - Preferem-se comunicações não bloqueantes para permitir overlap (comunicação + cálculo).

 - Requer controlo de fluxo para evitar saturação ou bloqueios entre etapas.
 
### Comparação geral

| Dimensão     | Tipo de paralelismo | Comunicação dominante | Bloqueio       | Persistência / Fluxo | Exemplo típico                         |
| ------------ | ------------------- | --------------------- | -------------- | -------------------- | -------------------------------------- |
| **Espacial** | De dados            | Coletiva              | Bloqueante     | Etapas sincronizadas | Processamento de matriz / simulação 2D |
| **Temporal** | De tarefas          | Ponto-a-ponto         | Não bloqueante | Fluxo contínuo       | Pipeline de áudio ou vídeo             |


In [38]:
!pwd

/projects/F202500003HPCVLABUTAD/ccvca/notebooks/a_exemplos_mpi


In [39]:
!sbatch job_espacial.sh

Submitted batch job 581079


In [ ]:
!sacctmgr show user $USER

In [7]:
!billing

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Account                     ┃ Used (h) ┃ Limit (h) ┃ Used (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ f202400002voucherdeucaliona │      437 │     50000 │     0.87 │
│ f202400002voucherdeucalionx │    95701 │    100000 │    95.70 │
│ f202500003hpcvlabutada      │      282 │     50000 │     0.57 │
│ f202500003hpcvlabutadg      │      277 │      1000 │    27.75 │
│ f202500003hpcvlabutadx      │    17122 │     50000 │    34.24 │
└─────────────────────────────┴──────────┴───────────┴──────────┘


In [8]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         43 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  256
  On-line CPU(s) list:   0-255
Vendor ID:               AuthenticAMD
  Model name:            AMD EPYC 7742 64-Core Processor
    CPU family:          23
    Model:               49
    Thread(s) per core:  2
    Core(s) per socket:  64
    Socket(s):           2
    Stepping:            0
    Frequency boost:     enabled
    CPU(s) scaling MHz:  88%
    CPU max MHz:         2250.0000
    CPU min MHz:         1500.0000
    BogoMIPS:            4500.04
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od nopl nonstop_tsc cpuid extd_apicid aperfmperf pni pc
                     

In [9]:
!nproc

256
